# Using SDx on the command line

## Introduction

This lab guides you through the steps involved in using SDx on the command line to build and application, and perform software and hardware emulation, and test the final design in hardware.

The lab will use the same vector addition source files as the previous SDx introductory lab.


## Objectives

After completing this lab, you will be able to:

* Run Software Emulation to verify the functionality of a design using a Makefile flow
* Run Hardware Emulation to verify the functionality including kernel hardware using a Makefile flow
* Build the full system and verify functionality in hardware on an AWS F1 instance


### Jupyter 
This document is a Jupyter notebook. A Jupyter notebook is an executable document. To run the tutorial, you can copy and paste the instructions in this document into your terminal and execute them, or if you use Jupyter Notebook, you can open this document and execute the cells directly from Jupyter. 
For more information and instructions on how to get Jupyter, see https://jupyter.org
The commands you will need to run are inside Jupyter code cells. Each command is prefixed with **!** which means pass the command to a shell. 
If you want to run the commands in a terminal, copy the command without the "!".

## Steps

### Check environment

First check XILINX_SDK and XILINX_XRT have been set in the current environment

In [1]:
!echo $XILINX_SDX
!echo $XILINX_VIVADO
!echo $XILINX_XRT

/opt/xilinx/SDx/2018.3
/opt/xilinx/Vivado/2018.3
/opt/xilinx/xrt


## Build and Run Emulation Flows

As you saw in the previous labs, the emulation modes allow testing of the design before doign a full hardware build. 

### Set build configuration

In previous labs, the *build configuration* (SW Emulation, HW Emulation, or System) was set from the GUI. When using the command line, this needs to be set with an environment variable XCL_EMULATION_MODE. 

This affects the build, and also affects the behaviour of an SDx executable when it is call as you will see later.

System Build is the default if no environment variable is set. 

* Set the XCL_EMULATION_MODE mode to Software Emulation

In [2]:
%set_env XCL_EMULATION_MODE=sw_emu

env: XCL_EMULATION_MODE=sw_emu


# Set the target platform
You will need to pass the path to your target platform when building your design. 

* Set and environment variable for your platform now. 

(Change the path below to point to your installed platform.)

In [3]:
%set_env U200_PLATFORM=/opt/xilinx/platforms/xilinx_u200_xdma_201830_1/xilinx_u200_xdma_201830_1.xpfm

env: U200_PLATFORM=/opt/xilinx/platforms/xilinx_u200_xdma_201830_1/xilinx_u200_xdma_201830_1.xpfm


## Compile OpenCL kernel

xocc is the SDx (Xilinx) OpenCL kernel. You will now compile the OpenCL kernel, in SW Emulation mode, targetting a specific platform. 

- Switches
    * **--target** specifies the Build mode
    * **-c** compile
    * **--platform** specifies the Platform
    * **-k** specifies the kerner name
    * **-g** will include debugging information
    * **-I** is the include directory (for including header, or other source files)
    * **-o** specifies the output file

For more information on the xocc compiler, see: 
https://www.xilinx.com/html_docs/xilinx2018_3/sdaccel_doc/wrj1504034328013.html

* Run xocc to compile the kernel

In [4]:
!xocc --target $XCL_EMULATION_MODE \
-c --platform $U200_PLATFORM \
-k krnl_vadd -g \
-I"./src" -o "kernel/krnl_vadd.xo" \
"./src/krnl_vadd.cl"


****** xocc v2018.3 (64-bit)
  **** SW Build 2416831 on Sat Dec 22 10:26:55 MST 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

Attempting to get a license: ap_opencl
Feature available: ap_opencl
INFO: [XOCC 60-1306] Additional information associated with this xocc compile can be found at:
	Reports: /home/nimbix/awslabs/sources/helloworld_ocl/_x/reports/krnl_vadd
	Log files: /home/nimbix/awslabs/sources/helloworld_ocl/_x/logs/krnl_vadd
INFO: [XOCC 60-585] Compiling for software emulation target
Running SDx Rule Check Server on port:45717
INFO: [XOCC 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u200_xdma_201830_1/xilinx_u200_xdma_201830_1.xpfm
INFO: [XOCC 60-423]   Target device: xilinx_u200_xdma_201830_1
INFO: [XOCC 60-242] Creating kernel: 'krnl_vadd'
INFO: [XOCC 60-594] Finished kernel compilation
INFO: [XOCC 60-586] Created kernel/krnl_vadd.xo
INFO: [XOCC 60-791] Total elapsed time: 0h 0m 9s


### Create the FPGA binary

- Switches
   * **-l** links one or more kernels into the platform to create the binary container
   * **--nk** specifies the number of compute units
   * **--xp** passes parameters to the Vivado tools. For example, to configure optimization or placement settings. 


* Run xocc to create the FPGA binary

In [5]:
!xocc -t $XCL_EMULATION_MODE \
--platform $U200_PLATFORM \
-l --nk krnl_vadd:1:krnl_vadd_1 \
-g --xp misc:solution_name=link -o"binary_container_1.xclbin" \
kernel/krnl_vadd.xo


****** xocc v2018.3 (64-bit)
  **** SW Build 2416831 on Sat Dec 22 10:26:55 MST 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

Attempting to get a license: ap_opencl
Feature available: ap_opencl
INFO: [XOCC 60-1306] Additional information associated with this xocc link can be found at:
	Reports: /home/nimbix/awslabs/sources/helloworld_ocl/_x/reports/link
	Log files: /home/nimbix/awslabs/sources/helloworld_ocl/_x/logs/link
INFO: [XOCC 60-629] Linking for software emulation target
Running SDx Rule Check Server on port:45605
INFO: [XOCC 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u200_xdma_201830_1/xilinx_u200_xdma_201830_1.xpfm
INFO: [XOCC 60-423]   Target device: xilinx_u200_xdma_201830_1
INFO: [XOCC 60-586] Created binary_container_1.xclbin
INFO: [XOCC 60-1307] Run completed. Additional information can be found in:
	Guidance: /home/nimbix/awslabs/sources/helloworld_ocl/_x/reports/link/link_guidance.html
	Steps Log File: /home/nimbix/awslabs/sources/hell

# Compile host application code

The host application code is C++ and is compiled with the Xilinx C++ compiler *xcpp*. 

- Switches
    * **-DSDX_PLATFORM** specifices the platfrom
    * **-g, -c, -std, -o** are all standard C++ compiler switches
    
* Compile the vadd.cpp file 


In [6]:
!xcpp -DSDX_PLATFORM=xilinx_u200_xdma_201830_1 -D__USE_XOPEN2K8 \
-I$XILINX_SDX/runtime/include/1_2/ \
-I$XILINX_VIVADO/include/ \
-g -c -std=c++14 \
-o "./vadd.o" \
"./src/vadd.cpp"

* Compile the xcl.cpp file

In [7]:
!xcpp -DSDX_PLATFORM=xilinx_u200_xdma_201830_1 \
-D__USE_XOPEN2K8 -I$XILINX_SDK/runtime/include/1_2/ \
-I$XILINX_VIVADO/include/ -g -c -std=c++14 \
-o "./xcl.o" \
"./src/xcl.cpp"

./src/xcl.cpp: In function 'xcl_world xcl_world_single(cl_device_type, const char*, const char*)':
./src/xcl.cpp:203:24: warning: '_cl_command_queue* clCreateCommandQueue(cl_context, cl_device_id, cl_command_queue_properties, cl_int*)' is deprecated [-Wdeprecated-declarations]
  world.command_queue = clCreateCommandQueue(world.context,
                        ^
In file included from /usr/include/CL/opencl.h:42:0,
                 from ./src/xcl.h:50,
                 from ./src/xcl.cpp:53:
/usr/include/CL/cl.h:1359:1: note: declared here
 clCreateCommandQueue(cl_context                     /* context */,
 ^
./src/xcl.cpp:203:24: warning: '_cl_command_queue* clCreateCommandQueue(cl_context, cl_device_id, cl_command_queue_properties, cl_int*)' is deprecated [-Wdeprecated-declarations]
  world.command_queue = clCreateCommandQueue(world.context,
                        ^
In file included from /usr/include/CL/opencl.h:42:0,
                 from ./src/xcl.h:50,
                 from ./src/x

In [8]:
#!/bin/bash /opt/xilinx/SDx/2018.3/settings64.sh

### Run the Linker

The host application files, and the OpenCL kernels have now been compiled. THe last step is to run the linker.

- Switches
    * -lxilinxopencl 
    * -lpthread 
    * -lrt 
    * -lstdc++ 
    * -Wl
    * -rpath
    * -lhlsmc++-GCC46
    * -lgmp 
    * -lmpfr 
    * -lIp_floating_point_v7_0_bitacc_cmodel

* Run the linker

# Check duplicate switches

In [9]:
!xcpp -o "vector_addition.exe" vadd.o xcl.o -lxilinxopencl -lpthread -lrt -lstdc++ \
-L$XILINX_SDK/runtime/lib/x86_64 \
-L$XILINX_XRT/lib/ \
-lxilinxopencl -lpthread -lrt -lstdc++ \
-Wl,-rpath,$XILINX_VIVADO/lnx64/lib/csim \
-L $XILINX_VIVADO/lnx64/lib/csim \
-lhlsmc++-GCC46 -Wl,-rpath,$XILINX_VIVADO/lnx64/tools/fpo_v7_0 \
-L /$XILINX_VIVADO/lnx64/tools/fpo_v7_0 \
-lgmp -lmpfr -lIp_floating_point_v7_0_bitacc_cmodel

### Emulation configuration file

An Enulation configuration file is required prior to running SW or HW Emulation. 
This provides details of the *platform* and is used by the emulation runtime. 

- Switches
    * **--nd** specifies number of devices. The default is 1
    
https://www.xilinx.com/html_docs/xilinx2018_3/sdaccel_doc/pnb1504034327301.html

* Build the emulation file

In [10]:
!emconfigutil --platform $U200_PLATFORM --nd 1


****** configutil v2018.3 (64-bit)
  **** SW Build 2416831 on Sat Dec 22 10:26:55 MST 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

INFO: [ConfigUtil 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u200_xdma_201830_1/xilinx_u200_xdma_201830_1.xpfm
emulation configuration file `emconfig.json` is created in current working directory 


## Test the application

The final step is to test the application.

Notice that when calling the executable, no information about the "mode" is specified. The executable runs in emulation or on hardware, based on the XCL_EMULATION_MODE environment variable specified earlier. 

In [11]:
!./vector_addition.exe ./binary_container_1.xclbin 10

Init arrays
Get Xilinx platform
Found: xilinx_u200_xdma_201830_1
Loading: './binary_container_1.xclbin'
10 + 10 = 20
11 + 11 = 22
12 + 12 = 24
13 + 13 = 26
14 + 14 = 28
15 + 15 = 30
16 + 16 = 32
17 + 17 = 34
18 + 18 = 36
19 + 19 = 38
20 + 20 = 40
21 + 21 = 42
22 + 22 = 44
23 + 23 = 46
24 + 24 = 48
25 + 25 = 50
26 + 26 = 52
27 + 27 = 54
28 + 28 = 56
29 + 29 = 58
30 + 30 = 60
31 + 31 = 62
32 + 32 = 64
33 + 33 = 66
34 + 34 = 68
35 + 35 = 70
36 + 36 = 72
37 + 37 = 74
38 + 38 = 76
39 + 39 = 78
40 + 40 = 80
41 + 41 = 82
42 + 42 = 84
43 + 43 = 86
44 + 44 = 88
45 + 45 = 90
46 + 46 = 92
47 + 47 = 94
48 + 48 = 96
49 + 49 = 98
50 + 50 = 100
51 + 51 = 102
52 + 52 = 104
53 + 53 = 106
54 + 54 = 108
55 + 55 = 110
56 + 56 = 112
57 + 57 = 114
58 + 58 = 116
59 + 59 = 118
60 + 60 = 120
61 + 61 = 122
62 + 62 = 124
63 + 63 = 126
64 + 64 = 128
65 + 65 = 130
66 + 66 = 132
67 + 67 = 134
68 + 68 = 136
69 + 69 = 138
70 + 70 = 140
71 + 71 = 142
72 + 72 = 144
73 + 73 = 146
74 + 74 = 148
75 + 75 = 150
76 + 76 = 15

## Build the System

If you would like to implement the kernel in hardware, clear the XCL_EMULATION_MODE environment variable, and rerun the notebook. Note that it takes around 2 hours to build the hardware. 

## Cleanup intermediate files

When you are finished, you can remove intermediate files. 

In [12]:
!rm *.o *.log *.info  emconfig.json *.str
!rm -rf .Xil _x 
!dir

Makefile		   kernel		 vector_addition.exe
binary_container_1.xclbin  sdaccel_cmd_line.txt  xclbin
command_line.ipynb	   src
